In [0]:
# Configurações de conexão com o banco PostgreSQL
jdbc_url = "jdbc:postgresql://dpg-cuciu352ng1s73b52bgg-a.ohio-postgres.render.com:5432/transactions_yz0m"
jdbc_properties = {
    "user": "transactions_yz0m_user",
    "password": "66BsNmXaNh3btK1ZY8GLbnrFmwUFK9xY",
    "driver": "org.postgresql.Driver"
}

# Nome da tabela no PostgreSQL
postgres_table = "transactions"

# Nome da tabela gerenciada no Databricks
databricks_table = "transactions"

# Carregar os dados do PostgreSQL para um DataFrame Spark
transactions_df = (
    spark.read.format("jdbc")
    .option("url", jdbc_url)
    .option("dbtable", postgres_table)
    .option("user", jdbc_properties["user"])
    .option("password", jdbc_properties["password"])
    .option("driver", jdbc_properties["driver"])
    .load()
)

# Exibir amostra dos dados carregados
transactions_df.show(5)

# Salvar os dados diretamente em uma tabela gerenciada no Databricks
transactions_df.write.format("delta").mode("overwrite").saveAsTable(databricks_table)

print(f"Tabela '{databricks_table}' criada com sucesso no Databricks.")


In [0]:
# Configurações de conexão com o banco PostgreSQL
jdbc_url = "jdbc:postgresql://dpg-cuciu352ng1s73b52bgg-a.ohio-postgres.render.com:5432/transactions_yz0m"
jdbc_properties = {
    "user": "transactions_yz0m_user",
    "password": "66BsNmXaNh3btK1ZY8GLbnrFmwUFK9xY",
    "driver": "org.postgresql.Driver"
}

# Nome da tabela no PostgreSQL e no Databricks
postgres_table = "transactions"
databricks_table = "transactions"

# 1. Obter o último `transaction_time` da tabela gerenciada no Databricks
last_transaction_time = spark.sql(f"""
    SELECT MAX(transaction_time) AS last_transaction_time 
    FROM {databricks_table}
""").collect()[0]["last_transaction_time"]

# Verificar se há um `last_transaction_time` válido
if last_transaction_time is None:
    print("Nenhum dado encontrado na tabela gerenciada. Carregando todos os dados do PostgreSQL.")
    query = f"(SELECT * FROM {postgres_table}) AS t"
else:
    print(f"Última transação encontrada: {last_transaction_time}. Carregando apenas os dados novos.")
    query = f"""
        (SELECT * 
         FROM {postgres_table} 
         WHERE transaction_time > '{last_transaction_time}') AS t
    """

# 2. Carregar apenas os dados novos do PostgreSQL para um DataFrame Spark
new_transactions_df = (
    spark.read.format("jdbc")
    .option("url", jdbc_url)
    .option("dbtable", query)
    .option("user", jdbc_properties["user"])
    .option("password", jdbc_properties["password"])
    .option("driver", jdbc_properties["driver"])
    .load()
)

# Verificar se há dados novos para inserir
if new_transactions_df.count() > 0:
    print(f"Inserindo {new_transactions_df.count()} novos registros na tabela gerenciada.")
    
    # 3. Inserir os dados novos na tabela gerenciada
    new_transactions_df.write.format("delta").mode("append").saveAsTable(databricks_table)
    
    print("Novos dados inseridos com sucesso!")
else:
    print("Nenhum dado novo encontrado para inserção.")
